In [1]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import itertools

from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.utils import set_random_seed
from stable_baselines3.common.callbacks import BaseCallback, CheckpointCallback
import copy
import random, math
import os
import torch as th
from torch import nn
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor

/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/util.py:55: DeprecationWarning: jax.xla_computation is deprecated. Please use the AOT APIs; see https://jax.readthedocs.io/en/latest/aot.html. For example, replace xla_computation(f)(*xs) with jit(f).lower(*xs).compiler_ir('hlo'). See CHANGELOG.md for 0.4.30 for more examples.
  from jax import xla_computation as _xla_computation


In [2]:
import math
import numpy as np
import pexpect
import ctypes

class Connect4Solver:
    !pip install stable_baselines3
    !git clone https://github.com/TonyCongqianWang/connect4_solver_fork.git && cd connect4_solver_fork && make
    !curl -L https://github.com/PascalPons/connect4/releases/download/book/7x6.book --output 7x6.book
    solver_path='./connect4_solver_fork/c4solver_c_interface.so'
    solver_lib = ctypes.CDLL(solver_path)
            
    solver_lib.solver_init.argtypes = [ctypes.c_char_p]
    solver_lib.solver_init.restype = ctypes.POINTER(ctypes.c_void_p)
    
    solver_lib.solver_delete.argtypes = [ctypes.POINTER(ctypes.c_void_p)]
    solver_lib.solver_delete.restype = None
    
    solver_lib.solver_solve.argtypes = [ctypes.POINTER(ctypes.c_void_p), ctypes.c_char_p, ctypes.c_bool, ctypes.c_bool, ctypes.c_char_p, ctypes.c_size_t]
    solver_lib.solver_solve.restype = ctypes.c_char_p
    def __init__(self):
        """
        Initializes the Connect4Solver with the path to the solver executable.

        Args:
            solver_path (str): Path to the Connect4 solver executable.
        """
        self.MAX_SCORE = 24
        self.handle = Connect4Solver.solver_lib.solver_init(None)
        self.result_buffer = ctypes.create_string_buffer(256)

    def __del__(self):
        """
        Destructor that sends EOF to the solver process.
        """
        if hasattr(self, 'child') and self.child is not None:
            try:
                self.child.sendeof()
            except:
                pass

    def _process_output(self, prompt_str, answer_str):
        """
        Processes the output from the solver.

        Args:
            prompt_str (str): The prompt string.
            answer_str (str): The answer string.

        Returns:
            list: List of floats representing the processed output.
        """
        if answer_str.startswith(prompt_str):
            answer_str = answer_str[len(prompt_str):].strip()
            
        answer_list = [float(x) for x in answer_str.split()]
        return answer_list

    def _softmax(self, x, temperature=1.0):
        """
        Calculates a modified softmax that approaches argmax for small temperatures.

        For very small temperatures, indices with the maximum value will receive
        equal probability, and the rest will receive 0.

        Args:
            x (list): List of values.
            temperature (float): Temperature parameter for softmax.

        Returns:
            list: List of probabilities.
        """
        if temperature <= 1e-5:  # Consider a very small temperature as argmax
            max_val = max(x)
            max_indices = [i for i, val in enumerate(x) if val == max_val]
            probabilities = [0.0] * len(x)
            prob = 1.0 / len(max_indices)
            for i in max_indices:
                probabilities[i] = prob
            return probabilities
        else:
            e_x = []
            for i in x:
                # Clipping to prevent overflow for large positive values
                exponent = i / temperature
                if exponent > 100:  # Or a suitable large value
                    e_x.append(float('inf'))
                elif exponent < -100:
                    e_x.append(0.0)
                else:
                    e_x.append(math.exp(exponent))

            sum_e_x = sum(e_x)
            if sum_e_x == 0:
                return ([1.0] * len(x)) / len(x)
            return [e / sum_e_x for e in e_x]

    def _transform_and_softmax(self, data, score_offset, temperature):
        """
        Transforms and calculates the softmax of the data.

        Args:
            data (list): List of data values.
            temperature (float): Temperature parameter for softmax.

        Returns:
            list: List of softmax probabilities.
        """
        transformed_data = []
        for x in data:
            sign = 1 if x > 0 else -1 if x < 0 else 0
            if x > -1000:
                transformed_x = sign * ((abs(x) + score_offset) / self.MAX_SCORE * 5)
            else:
                transformed_x = -1000
            transformed_data.append(transformed_x)
        #print(transformed_data)
        return self._softmax(transformed_data, temperature)

    def _random_index(self, softmax_probs):
        """
        Selects a random index based on softmax probabilities.

        Args:
            softmax_probs (list): List of softmax probabilities.

        Returns:
            int: Selected index.
        """
        selected_index = np.random.choice(len(softmax_probs), p=softmax_probs)
        return selected_index

    def get_solver_move(self, move_str, temperature=1.0):
        """
        Gets a move from the solver.

        Args:
            move_str (str): Move string to send to the solver.
            temperature (float): Temperature parameter for softmax.

        Returns:
            int: Selected move index.
        """
        try:
            result = Connect4Solver.solver_lib.solver_solve(self.handle, move_str.encode("utf-8"), False, True, self.result_buffer, 256)
            answer = result.decode()
            score_offset = math.floor(len(move_str) / 2)
            probas = self._transform_and_softmax(self._process_output(move_str, answer), score_offset, temperature)
            #print(f"{answer}")
            #print(probas)
            return self._random_index(probas)
        except Exception as e:
            print(f"{e}")
            print(f"{answer}")
        return 0

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Cloning into 'connect4_solver_fork'...
remote: Enumerating objects: 149, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 149 (delta 17), reused 13 (delta 13), pack-reused 122 (from 2)
Receiving objects: 100% (149/149), 67.77 KiB | 2.82 MiB/s, done.
Resolving deltas: 100% (87/87), done.
g++ --std=c++11 -W -Wall -O3 -DNDEBUG -fPIC -MM Solver.cpp > ./.depend
g++ --std=c++11 -W -Wall -O3 -DNDEBUG -fPIC   -c -o Solver.o Solver.cpp
g++ --std=c++11 -W -Wall -O3 -DNDEBUG -fPIC   -c -o main.o main.cpp
g++ --std=c++11 -W -Wall -O3 -DNDEBUG -fPIC  -o c4solver main.o Solver.o 
g++ --std=c++11 -W -Wall -O3 -DNDEBUG -fPIC   -c -o generator.o generator.cpp
g++ --std=c++11 -W -Wall -O3 -DNDEBUG -fPIC  -o generator generator.o 
g++ --std=c++11 -W -Wall -O3 -DNDEBUG -fPIC   -c -o solver_c_interface.o solver_c_interface.cpp
g++ --std=c++11 -W -Wall -O3 -DNDEBUG -fPIC  -shared -o c4solver_c_interface.so solver_c_interface.o Solver.o 
  % T

In [3]:
import zipfile

def zip_directories(directory_paths, working_dir):
    """
    Zips the given directories into their parent directory.

    Args:
        directory_paths (list): A list of paths to directories.

    Returns:
        list: A list of paths to the created zip files.
    """
    os.makedirs(working_dir, exist_ok=True)
    
    zip_file_paths = []
    for dir_path in directory_paths:
        if not os.path.isdir(dir_path):
            print(f"Warning: {dir_path} is not a directory. Skipping.")
            continue

        parent_dir = working_dir
        dir_name = os.path.basename(dir_path)
        zip_file_path = os.path.join(parent_dir, f"{dir_name}.zip")

        try:
            with zipfile.ZipFile(zip_file_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
                for root, _, files in os.walk(dir_path):
                    for file in files:
                        file_path = os.path.join(root, file)
                        relative_path = os.path.relpath(file_path, dir_path)
                        zipf.write(file_path, relative_path)
            zip_file_paths.append(zip_file_path)
        except Exception as e:
            print(f"Error zipping {dir_path}: {e}")

    return zip_file_paths

In [4]:
class ConnectFourEnv(gym.Env):
    metadata = {"render_modes": ["human", "ansi", "rgb_array"], "render_fps": 1}
    def __init__(self, render_mode=None, board_rows=6, board_cols=7):
        super(ConnectFourEnv, self).__init__()
        self.board_rows = board_rows
        self.board_cols = board_cols
        self.action_space = spaces.Discrete(self.board_cols)  # Columns to drop a piece
        self.observation_space = spaces.Box(low=0, high=255, shape=(2, self.board_rows, self.board_cols), dtype=np.uint8)  # two binary matrices. one for each players stones
        self.render_mode = render_mode
        self.move_history = ""
        
        self.reset()

    def reset(self, seed=None, options=None):
        super().reset(seed=seed, options=options)
        self.board = np.zeros((self.board_rows, self.board_cols), dtype=np.int8)
        self.player = 1  # Player 1 starts
        self.done = False
        self.winner = None
        self.turns = 0
        self.move_history_str = ""
        info = {}
        return self._get_observation(), info

    def step(self, action):
        if self.done:
            return self._get_observation(), 0, True, False, {}

        if not self._is_valid_move(action):
            return self._get_observation(), -50, False, False, {}

        self._drop_piece(action)
        self.move_history_str += str(action + 1)
        self.turns += 1

        if self._check_win():
            self.done = True
            self.winner = self.player
            reward = 80 + 20 * (len(self.board.flatten()) - self.turns) / len(self.board.flatten())
        elif self._check_draw():
            self.done = True
            reward = 0
        else:
            reward = 0
        self.player *= -1  # Switch players
        return  self._get_observation(), reward, self.done, False, {}

    def get_valid_moves(self):
        valid_moves = []
        for col in range(self.board_cols):
            if self._is_valid_move(col):
                valid_moves.append(col)
        return valid_moves

    def _get_observation(self):
        m, n = self.board.shape
        player_perspective = self.board * self.player
        new_array = np.zeros((2, m, n), dtype=np.uint8)
        new_array[0, :, :] = 255 * (player_perspective == 1).astype(np.uint8)
        new_array[1, :, :] = 255 * (player_perspective == -1).astype(np.uint8)
        return new_array

    def _is_valid_move(self, col):
        return self.board[0, col] == 0

    def _drop_piece(self, col):
        for row in range(self.board_rows - 1, -1, -1):
            if self.board[row, col] == 0:
                self.board[row, col] = self.player
                return

    def _check_win(self):
        # Check horizontal, vertical, and diagonal wins
        for r in range(self.board_rows):
            for c in range(self.board_cols - 3):
                if (
                    self.board[r, c] == self.board[r, c + 1] == self.board[r, c + 2] == self.board[r, c + 3] != 0
                ):
                    return True

        for c in range(self.board_cols):
            for r in range(self.board_rows - 3):
                if (
                    self.board[r, c] == self.board[r + 1, c] == self.board[r + 2, c] == self.board[r + 3, c] != 0
                ):
                    return True

        for r in range(self.board_rows - 3):
            for c in range(self.board_cols - 3):
                if (
                    self.board[r, c] == self.board[r + 1, c + 1] == self.board[r + 2, c + 2] == self.board[r + 3, c + 3] != 0
                ):
                    return True

        for r in range(3, self.board_rows):
            for c in range(self.board_cols - 3):
                if (
                    self.board[r, c] == self.board[r - 1, c + 1] == self.board[r - 2, c + 2] == self.board[r - 3, c + 3] != 0
                ):
                    return True
        return False

    def _check_draw(self):
        return np.all(self.board != 0)

    def render(self):
        board_str = ""
        board_str += "-" * (self.board_cols * 2 + 3) + "\n"
        for row in self.board:
            board_str += "| "
            for cell in row:
                if cell == 1:
                    board_str += "x "
                elif cell == -1:
                    board_str += "o "
                else:
                    board_str += "  "
            board_str += "|\n"
        board_str += "-" * (self.board_cols * 2 + 3)
        print(board_str)

In [5]:
from collections import defaultdict
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

def calculate_elo(rating_a, rating_b, score_a):
    """Calculates the new Elo ratings for two players."""
    k = 8
    q = 480
    expected_a = 1 / (1 + 10**((rating_b - rating_a) / q))
    expected_b = 1 / (1 + 10**((rating_a - rating_b) / q))
    
    new_rating_a = rating_a + k * (score_a - expected_a)
    new_rating_b = rating_b + k * ((1 - score_a) - expected_b)
    return new_rating_a, new_rating_b

def evaluate_agents(agent_paths, num_episodes=100):
    """Evaluates multiple trained PPO agents and calculates Elo scores."""
    env = ConnectFourEnv()
    print(f"{agent_paths=}")
    agents = [PPO.load(path) for path in agent_paths]
    elo_ratings = [1300] * len(agents)
    solver = Connect4Solver()
    solver_temps = [0.0, 0.1, 0.15, 0.2, 0.3, 0.5, 1.0]
    agents += solver_temps
    agent_names = [path.split("/")[-1] for path in agent_paths] + [f"Solver_{t}" for t in solver_temps]
    num_agents = len(agents)
    # Empirically determined ratings
    elo_ratings += [1970, 1910, 1840, 1820, 1710, 1500, 1140]
    
    results = defaultdict(lambda: defaultdict(lambda: {"wins_first": 0, "draws_first": 0, "loses_first": 0}))
    # Generate all possible matchups
    matchups = [(i, j) for i in range(num_agents) for j in range(num_agents)]  # Add self-play

    for episode in range(num_episodes):
        random.shuffle(matchups) #shuffle matchups for each episode.
        for agent1_index, agent2_index in matchups:
            agent1 = agents[agent1_index]
            agent2 = agents[agent2_index]

            agent1_wins_first = 0
            agent1_draws_first = 0
            agent1_loses_first = 0

            obs, _ = env.reset()
            done = False

            while not done:
                current_player = env.player
                if current_player == 1:
                    if type(agent1) == float:
                        action = solver.get_solver_move(env.move_history_str, agent1)
                    else:
                        action, _ = agent1.predict(obs, deterministic=False)
                else:
                    if type(agent2) == float:
                        action = solver.get_solver_move(env.move_history_str, agent2)
                    else:
                        action, _ = agent2.predict(obs, deterministic=False)

                valid_moves = env.get_valid_moves()
                if action not in valid_moves:
                    action = random.choice(valid_moves)
                obs, reward, done, truncated, _ = env.step(action)

                if done or truncated:
                    if reward > 0:
                        if current_player == 1:
                            agent1_wins_first = 1
                        else:
                            agent1_loses_first = 1
                    else:
                        agent1_draws_first = 1
                    break

            # Calculate Elo. Don't change Elo for self-play or for solvers
            score_agent1 = (agent1_wins_first + 0.5 * agent1_draws_first)
            if agent1_index != agent2_index:
                new_rating_1, new_rating_2 = calculate_elo(
                    elo_ratings[agent1_index], elo_ratings[agent2_index], score_agent1
                )
                if not type(agents[agent1_index]) == float:
                    elo_ratings[agent1_index] = new_rating_1
                if not type(agents[agent2_index]) == float:
                    elo_ratings[agent2_index] = new_rating_2

            # Update results
            results[agent1_index][agent2_index]["wins_first"] += agent1_wins_first
            results[agent1_index][agent2_index]["draws_first"] += agent1_draws_first
            results[agent1_index][agent2_index]["loses_first"] += agent1_loses_first
        print(f"Episode {episode + 1} / 100 done.")

    env.close()

    sorted_agents = sorted(range(num_agents), key=lambda i: elo_ratings[i], reverse=True)

    columns = ["Agent", "Elo"] + [f"Agent {i}" for i in range(num_agents)]
    data = []

    for agent1_index in sorted_agents:
        agent_name = agent_names[agent1_index]
        row = [agent_name, f"{elo_ratings[agent1_index]:.2f}"]
        for agent2_index in sorted_agents:
            data_dict = results[agent1_index][agent2_index]
            row.append(f"{data_dict['wins_first']} / {data_dict['draws_first']} / {data_dict['loses_first']}")
        data.append(row)

    df = pd.DataFrame(data, columns=columns)
    return results, df

In [6]:
try:
    agent_dir = "/kaggle/input/connect-4-agents/"
    agent_files = [f for f in os.listdir(agent_dir)]
    agent_paths = [os.path.join(agent_dir, f) for f in agent_files]
    agent_paths = zip_directories(agent_paths, "/kaggle/working/agents")
except:
    agent_paths = []

In [7]:
table, df = evaluate_agents(agent_paths, num_episodes=100)

agent_paths=['/kaggle/working/agents/014d.zip', '/kaggle/working/agents/011b.zip', '/kaggle/working/agents/013a.zip', '/kaggle/working/agents/011d.zip', '/kaggle/working/agents/014e.zip', '/kaggle/working/agents/015a.zip', '/kaggle/working/agents/015d.zip', '/kaggle/working/agents/012e.zip', '/kaggle/working/agents/014b.zip', '/kaggle/working/agents/013d.zip', '/kaggle/working/agents/012c.zip', '/kaggle/working/agents/012b.zip', '/kaggle/working/agents/013h.zip', '/kaggle/working/agents/013e.zip', '/kaggle/working/agents/011a.zip', '/kaggle/working/agents/015c.zip', '/kaggle/working/agents/015b.zip', '/kaggle/working/agents/012d.zip', '/kaggle/working/agents/012a.zip', '/kaggle/working/agents/014c.zip', '/kaggle/working/agents/013c.zip', '/kaggle/working/agents/013g.zip', '/kaggle/working/agents/013b.zip', '/kaggle/working/agents/011e.zip', '/kaggle/working/agents/013f.zip', '/kaggle/working/agents/014a.zip', '/kaggle/working/agents/011c.zip']


/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/save_util.py:437: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  th_object = th.load(file_content, map_locati

Episode 1 / 100 done.
Episode 2 / 100 done.
Episode 3 / 100 done.
Episode 4 / 100 done.
Episode 5 / 100 done.
Episode 6 / 100 done.
Episode 7 / 100 done.
Episode 8 / 100 done.
Episode 9 / 100 done.
Episode 10 / 100 done.
Episode 11 / 100 done.
Episode 12 / 100 done.
Episode 13 / 100 done.
Episode 14 / 100 done.
Episode 15 / 100 done.
Episode 16 / 100 done.
Episode 17 / 100 done.
Episode 18 / 100 done.
Episode 19 / 100 done.
Episode 20 / 100 done.
Episode 21 / 100 done.
Episode 22 / 100 done.
Episode 23 / 100 done.
Episode 24 / 100 done.
Episode 25 / 100 done.
Episode 26 / 100 done.
Episode 27 / 100 done.
Episode 28 / 100 done.
Episode 29 / 100 done.
Episode 30 / 100 done.
Episode 31 / 100 done.
Episode 32 / 100 done.
Episode 33 / 100 done.
Episode 34 / 100 done.
Episode 35 / 100 done.
Episode 36 / 100 done.
Episode 37 / 100 done.
Episode 38 / 100 done.
Episode 39 / 100 done.
Episode 40 / 100 done.
Episode 41 / 100 done.
Episode 42 / 100 done.
Episode 43 / 100 done.
Episode 44 / 100 don

In [8]:
df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Agent,Elo,Agent 0,Agent 1,Agent 2,Agent 3,Agent 4,Agent 5,Agent 6,Agent 7,Agent 8,Agent 9,Agent 10,Agent 11,Agent 12,Agent 13,Agent 14,Agent 15,Agent 16,Agent 17,Agent 18,Agent 19,Agent 20,Agent 21,Agent 22,Agent 23,Agent 24,Agent 25,Agent 26,Agent 27,Agent 28,Agent 29,Agent 30,Agent 31,Agent 32,Agent 33
0,Solver_0.0,1970.00,100 / 0 / 0,100 / 0 / 0,100 / 0 / 0,100 / 0 / 0,100 / 0 / 0,100 / 0 / 0,100 / 0 / 0,100 / 0 / 0,100 / 0 / 0,100 / 0 / 0,100 / 0 / 0,100 / 0 / 0,100 / 0 / 0,100 / 0 / 0,100 / 0 / 0,100 / 0 / 0,100 / 0 / 0,100 / 0 / 0,100 / 0 / 0,100 / 0 / 0,100 / 0 / 0,100 / 0 / 0,100 / 0 / 0,100 / 0 / 0,100 / 0 / 0,100 / 0 / 0,100 / 0 / 0,100 / 0 / 0,100 / 0 / 0,100 / 0 / 0,100 / 0 / 0,100 / 0 / 0,100 / 0 / 0,100 / 0 / 0
1,Solver_0.1,1910.00,78 / 19 / 3,83 / 14 / 3,83 / 17 / 0,87 / 12 / 1,86 / 14 / 0,91 / 9 / 0,95 / 5 / 0,98 / 2 / 0,90 / 10 / 0,94 / 6 / 0,94 / 6 / 0,95 / 5 / 0,95 / 5 / 0,95 / 5 / 0,89 / 11 / 0,89 / 11 / 0,86 / 14 / 0,84 / 16 / 0,94 / 6 / 0,91 / 8 / 1,98 / 2 / 0,88 / 12 / 0,94 / 6 / 0,91 / 9 / 0,95 / 5 / 0,96 / 4 / 0,92 / 8 / 0,90 / 10 / 0,99 / 1 / 0,91 / 9 / 0,95 / 5 / 0,96 / 4 / 0,94 / 6 / 0,98 / 2 / 0
2,Solver_0.15,1840.00,56 / 29 / 15,62 / 30 / 8,66 / 29 / 5,66 / 28 / 6,84 / 13 / 3,84 / 15 / 1,92 / 8 / 0,91 / 8 / 1,93 / 6 / 1,87 / 13 / 0,88 / 11 / 1,85 / 15 / 0,85 / 15 / 0,83 / 16 / 1,90 / 10 / 0,85 / 14 / 1,85 / 15 / 0,85 / 15 / 0,89 / 11 / 0,91 / 9 / 0,91 / 9 / 0,84 / 16 / 0,83 / 17 / 0,88 / 12 / 0,82 / 18 / 0,86 / 13 / 1,79 / 20 / 1,87 / 13 / 0,86 / 14 / 0,94 / 6 / 0,97 / 3 / 0,93 / 7 / 0,92 / 8 / 0,100 / 0 / 0
3,Solver_0.2,1820.00,47 / 36 / 17,40 / 38 / 22,49 / 29 / 22,50 / 30 / 20,74 / 19 / 7,81 / 19 / 0,86 / 13 / 1,79 / 20 / 1,88 / 7 / 5,88 / 12 / 0,77 / 22 / 1,79 / 18 / 3,81 / 16 / 3,89 / 11 / 0,83 / 15 / 2,79 / 18 / 3,80 / 18 / 2,80 / 19 / 1,92 / 8 / 0,85 / 14 / 1,92 / 8 / 0,81 / 18 / 1,89 / 10 / 1,88 / 12 / 0,87 / 13 / 0,86 / 13 / 1,71 / 28 / 1,80 / 20 / 0,81 / 18 / 1,88 / 11 / 1,96 / 3 / 1,88 / 12 / 0,94 / 5 / 1,96 / 3 / 1
4,Solver_0.3,1710.00,25 / 17 / 58,20 / 34 / 46,38 / 24 / 38,42 / 20 / 38,42 / 29 / 29,78 / 16 / 6,77 / 14 / 9,77 / 14 / 9,65 / 15 / 20,80 / 9 / 11,69 / 25 / 6,78 / 14 / 8,75 / 16 / 9,74 / 14 / 12,70 / 21 / 9,75 / 20 / 5,77 / 17 / 6,75 / 18 / 7,78 / 12 / 10,77 / 12 / 11,84 / 13 / 3,76 / 17 / 7,78 / 15 / 7,84 / 11 / 5,83 / 15 / 2,79 / 11 / 10,74 / 18 / 8,75 / 18 / 7,82 / 13 / 5,88 / 8 / 4,87 / 9 / 4,83 / 13 / 4,81 / 15 / 4,91 / 2 / 7
5,015b.zip,1572.19,14 / 27 / 59,16 / 17 / 67,13 / 24 / 63,15 / 22 / 63,20 / 14 / 66,43 / 23 / 34,45 / 41 / 14,8 / 16 / 76,38 / 5 / 57,17 / 43 / 40,15 / 77 / 8,47 / 13 / 40,59 / 9 / 32,78 / 4 / 18,93 / 0 / 7,96 / 1 / 3,63 / 5 / 32,95 / 0 / 5,54 / 4 / 42,57 / 11 / 32,24 / 2 / 74,38 / 16 / 46,54 / 17 / 29,49 / 4 / 47,50 / 17 / 33,64 / 2 / 34,78 / 2 / 20,59 / 27 / 14,43 / 7 / 50,53 / 3 / 44,73 / 8 / 19,27 / 33 / 40,50 / 8 / 42,68 / 4 / 28
6,014a.zip,1516.47,9 / 30 / 61,13 / 23 / 64,10 / 25 / 65,14 / 21 / 65,24 / 16 / 60,8 / 2 / 90,18 / 3 / 79,43 / 7 / 50,45 / 6 / 49,33 / 0 / 67,11 / 1 / 88,43 / 4 / 53,49 / 21 / 30,21 / 0 / 79,28 / 0 / 72,34 / 1 / 65,67 / 2 / 31,46 / 2 / 52,69 / 2 / 29,63 / 6 / 31,92 / 0 / 8,31 / 7 / 62,21 / 7 / 72,57 / 0 / 43,91 / 2 / 7,68 / 1 / 31,54 / 1 / 45,57 / 25 / 18,71 / 3 / 26,59 / 2 / 39,70 / 4 / 26,20 / 11 / 69,53 / 13 / 34,71 / 3 / 26
7,015c.zip,1503.97,21 / 26 / 53,15 / 21 / 64,15 / 19 / 66,16 / 19 / 65,15 / 16 / 69,19 / 4 / 77,56 / 15 / 29,23 / 3 / 74,39 / 5 / 56,51 / 9 / 40,35 / 44 / 21,53 / 19 / 28,39 / 4 / 57,84 / 16 / 0,76 / 3 / 21,96 / 3 / 1,86 / 2 / 12,66 / 11 / 23,60 / 2 / 38,79 / 7 / 14,53 / 4 / 43,49 / 16 / 35,60 / 25 / 15,51 / 34 / 15,25 / 18 / 57,84 / 4 / 12,95 / 1 / 4,63 / 24 / 13,48 / 5 / 47,79 / 3 / 18,69 / 11 / 20,56 / 24 / 20,50 / 32 / 18,76 / 1 / 23
8,Solver_0.5,1500.00,9 / 12 / 79,12 / 13 / 75,9 / 10 / 81,12 / 7 / 81,23 / 18 / 59,62 / 10 / 28,68 / 10 / 22,58 / 8 / 34,54 / 7 / 39,68 / 7 / 25,68 / 8 / 24,56 / 10 / 34,64 / 7 / 29,66 / 6 / 28,58 / 11 / 31,71 / 6 / 23,64 / 7 / 29,68 / 5 / 27,60 / 12 / 28,63 